In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon_EnetV2_OpenMax/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [2]:
from networks.resnet_big import ConResNet
from networks.efficient_net import ConEfficientNet
from networks.efficient_net_lite import ConEfficientNet as ConEfficientNetLite
from networks.efficient_net_b0 import ConEfficientNet as ConEfficientNetB0
from networks.vision_trans import CEViT
from networks.rec_cnn import RecCNN
from torchsummary import summary
from thop import profile
import torch
import torch.nn as nn 

In [19]:
import time
import torch

def process_frame(model, frame_data, device):
    start_time = time.time()
    
    model.eval()
    frame_data = frame_data.to(device)
    
    with torch.no_grad(): 
        model(frame_data)
        
    end_time = time.time()
    processing_time = (end_time - start_time) * 1000
    return processing_time


In [4]:
conenet_model = ConEfficientNet(embedding_dim=1792, feat_dim=128, head='mlp', pretrained=False)

In [6]:
print(summary(conenet_model, input_size=(3, 64, 64), device='cpu'))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 48, 64, 64]           1,296
       BatchNorm2d-2           [-1, 48, 64, 64]              96
MemoryEfficientSwish-3           [-1, 48, 64, 64]               0
         ZeroPad2d-4           [-1, 48, 66, 66]               0
Conv2dStaticSamePadding-5           [-1, 48, 64, 64]             432
       BatchNorm2d-6           [-1, 48, 64, 64]              96
MemoryEfficientSwish-7           [-1, 48, 64, 64]               0
          Identity-8             [-1, 48, 1, 1]               0
Conv2dStaticSamePadding-9             [-1, 12, 1, 1]             588
MemoryEfficientSwish-10             [-1, 12, 1, 1]               0
         Identity-11             [-1, 12, 1, 1]               0
Conv2dStaticSamePadding-12             [-1, 48, 1, 1]             624
         Identity-13           [-1, 48, 64, 64]               0
Conv2dStaticSame

In [7]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(conenet_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.padding.ZeroPad2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
#Parameters (M): 6.78
MACs (M): 34.19


In [36]:

conenet_lite_model = ConEfficientNetLite(embedding_dim=1024, feat_dim=128, head='mlp', pretrained=False)
# conenet_model = conenet_model.to('cuda')

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conenet_model = conenet_lite_model.to(device)
frame_data = torch.rand(1, 3, 64, 64)

processing_time = process_frame(conenet_model, frame_data, device)

print(f"Thời gian xử lý một frame: {processing_time:.1f} ms")

Thời gian xử lý một frame: 7.5 ms


In [19]:
summary(conenet_lite_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
             ReLU6-3           [-1, 32, 64, 64]               0
         ZeroPad2d-4           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-5           [-1, 32, 64, 64]             288
       BatchNorm2d-6           [-1, 32, 64, 64]              64
             ReLU6-7           [-1, 32, 64, 64]               0
          Identity-8           [-1, 32, 64, 64]               0
Conv2dStaticSamePadding-9           [-1, 16, 64, 64]             512
      BatchNorm2d-10           [-1, 16, 64, 64]              32
      MBConvBlock-11           [-1, 16, 64, 64]               0
         Identity-12           [-1, 16, 64, 64]               0
Conv2dStaticSamePadding-13           [-1, 96, 64, 64]           1,536
      BatchNorm2d-14   

In [10]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(conenet_lite_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.padding.ZeroPad2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
#Parameters (M): 2.54
MACs (M): 14.83


In [16]:
# model = ConResNet()
conenet_b0_model = ConEfficientNetB0(embedding_dim=1280, feat_dim=128, head='mlp', pretrained=False)
# conenet_model = conenet_model.to('cuda')

In [17]:
summary(conenet_b0_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
MemoryEfficientSwish-3           [-1, 32, 64, 64]               0
         ZeroPad2d-4           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-5           [-1, 32, 64, 64]             288
       BatchNorm2d-6           [-1, 32, 64, 64]              64
MemoryEfficientSwish-7           [-1, 32, 64, 64]               0
          Identity-8             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-9              [-1, 8, 1, 1]             264
MemoryEfficientSwish-10              [-1, 8, 1, 1]               0
         Identity-11              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-12             [-1, 32, 1, 1]             288
         Identity-13           [-1, 32, 64, 64]               0
Conv2dStaticSame

In [18]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(conenet_b0_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.padding.ZeroPad2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
#Parameters (M): 3.49
MACs (M): 15.78


In [30]:
vit_model = CEViT(emb_size=256, n_classes=10)

In [31]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(vit_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
#Parameters (M): 9.68
MACs (M): 164.00


In [46]:
summary(vit_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 256, 4, 4]         196,864
         Rearrange-2              [-1, 16, 256]               0
    PatchEmbedding-3              [-1, 17, 256]               0
         LayerNorm-4              [-1, 17, 256]             512
            Linear-5              [-1, 17, 768]         197,376
           Dropout-6            [-1, 8, 17, 17]               0
            Linear-7              [-1, 17, 256]          65,792
MultiHeadAttention-8              [-1, 17, 256]               0
           Dropout-9              [-1, 17, 256]               0
      ResidualAdd-10              [-1, 17, 256]               0
        LayerNorm-11              [-1, 17, 256]             512
           Linear-12             [-1, 17, 1024]         263,168
             GELU-13             [-1, 17, 1024]               0
          Dropout-14             [-1, 1

In [40]:
rec_cnn_model = RecCNN(num_classes=10)

In [43]:
x = torch.rand((1, 3, 64, 64))
macs, params = profile(rec_cnn_model, inputs=(x,))

print(f"#Parameters (M): {params / 1e6:.2f}")
print(f"MACs (M): {macs / 1e6:.2f}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
#Parameters (M): 2.47
MACs (M): 160.20


In [47]:
summary(rec_cnn_model, input_size=(3, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,792
         MaxPool2d-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 32, 32]          73,856
         MaxPool2d-4          [-1, 128, 16, 16]               0
            Conv2d-5          [-1, 256, 16, 16]         295,168
         MaxPool2d-6            [-1, 256, 8, 8]               0
 AdaptiveAvgPool2d-7            [-1, 256, 4, 4]               0
            Linear-8                  [-1, 512]       2,097,664
            Linear-9                   [-1, 10]           5,130
Total params: 2,473,610
Trainable params: 2,473,610
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 4.41
Params size (MB): 9.44
Estimated Total Size (MB): 13.89
--------------------------------------

In [8]:
from networks.convNeXt_tiny import ConvNeXtTinyMPNCOV

In [9]:
import torch

x = torch.randn(4, 3, 64, 64)
model = ConvNeXtTinyMPNCOV(num_classes=8, attention='Cov')
out = model(x)
print(out.shape)  # [4, 8]

torch.Size([4, 8])
